In [11]:
import torch
import numpy as np
import pandas as pd
device = torch.device("cpu")
#device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification, AdamW, get_scheduler
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')                                                     
#model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=658).to(device)
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
model_name = "sberbank-ai/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=655).to(device)
from sklearn import preprocessing
Label_encoder = preprocessing.LabelEncoder()
#Label_encoder.classes_ = np.load('./classes.npy', allow_pickle=True)
Label_encoder.classes_ = np.load('./cl_classes.npy', allow_pickle=True)
#model.load_state_dict(torch.load("../../best_model3004/pytorch_model.bin", map_location=device))
model.load_state_dict(torch.load("../../best_model2705/pytorch_model.bin", map_location=device))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [3]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [8]:
def predict_class(text):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        return Label_encoder.inverse_transform([predicted_class_id])

In [12]:
predict_class('Автомобильные покрышки резиновые новые')[0]

'4011'

In [4]:
def predict_prob(text, qtty=2):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    result = dict()
    p = torch.nn.functional.softmax(logits, dim=1)
    for i in range(qtty):
        a = p.argmax().item()
        result[Label_encoder.inverse_transform([a])[0]] = p[0][a].item()
        p[0][a] = 0
    return result

In [3]:
print(predict_prob('бананы в коробках', 5))

{'0808': 0.8409392833709717, '0805': 0.014152244664728642, '2007': 0.006393834948539734, '6702': 0.005769975483417511, '2106': 0.005318484269082546}


In [31]:
preds = predict_prob("гелевая ручка", 5)
print('Code', 'prob-ty', sep=':    ')
[print(i, f"{preds[i]:.{5}f}", sep=':    ') for i in preds]


Code:    prob-ty
8477:    0.27823
3926:    0.13464
6406:    0.03673
9608:    0.01928
3923:    0.01838


[None, None, None, None, None]

In [5]:
def consolelog(probs):
    print('Code', 'prob-ty', sep=':|    ')
    print('_________________')
    for each in probs:
        print(each, f"{probs[each]:.{5}f}", sep=':|    ')
        print('_________________')

In [16]:
consolelog(predict_prob("предмет для письма", 5))

Code:|    prob-ty
_________________
4820:|    0.08355
_________________
9609:|    0.04148
_________________
8304:|    0.03196
_________________
4911:|    0.02679
_________________
9017:|    0.02287
_________________
